<a href="https://colab.research.google.com/github/nidjosep/student-feedback-analysis/blob/master/GRU_Model_2_Actionable_Insights_from_Student_Feedback_Sentiment_Analysis_Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [386]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from google.colab import drive


# Download NLTK stopwords
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

# Function to remove stop words
def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

# Mount your Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/MCSC/TME_6015/Project/Emotion_final.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Mounted at /content/drive


In [387]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers  # Import the regularizers module
from tensorflow.keras.optimizers import Adam  # Import Adam optimizer
from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd


texts = df['Review']
labels = df['Label']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
encoded_labels = label_encoder.fit_transform(labels)

# To see the mapping of string labels to integers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping: ", label_mapping)

# Hyperparameters
vocab_size = 10000  # Number of unique words in the vocabulary
embedding_dim = 100  # Number of dimensions for each word vector
max_length = 100    # Max length of each input sequence
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"   # Token for out-of-vocabulary words

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(padded, encoded_labels, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(GRU(64, return_sequences=True)),
    Bidirectional(GRU(32)),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

# Using Adam optimizer with a different learning rate
optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_texts, train_labels, epochs=20, batch_size=64, validation_data=(val_texts, val_labels), callbacks=[early_stopping])

Label Mapping:  {'anger': 0, 'fear': 1, 'happy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}
Epoch 1/20
269/269 [==============================] - 27s 78ms/step - loss: 1.8302 - accuracy: 0.3819 - val_loss: 1.2953 - val_accuracy: 0.5736
Epoch 2/20
269/269 [==============================] - 7s 26ms/step - loss: 1.0509 - accuracy: 0.6538 - val_loss: 0.7953 - val_accuracy: 0.7642
Epoch 3/20
269/269 [==============================] - 6s 22ms/step - loss: 0.6119 - accuracy: 0.8220 - val_loss: 0.5526 - val_accuracy: 0.8350
Epoch 4/20
269/269 [==============================] - 6s 21ms/step - loss: 0.4265 - accuracy: 0.8837 - val_loss: 0.4911 - val_accuracy: 0.8548
Epoch 5/20
269/269 [==============================] - 5s 20ms/step - loss: 0.3181 - accuracy: 0.9192 - val_loss: 0.4941 - val_accuracy: 0.8530
Epoch 6/20
269/269 [==============================] - 5s 19ms/step - loss: 0.2596 - accuracy: 0.9367 - val_loss: 0.4582 - val_accuracy: 0.8770
Epoch 7/20
269/269 [=============================

In [389]:
def predict_sentiment(text):
    # Tokenize and pad the text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Predict
    prediction = model.predict(padded_sequence)
    sentiment = label_encoder.inverse_transform([np.argmax(prediction)])[0]

    return sentiment

# Example usage
sentence = "The course is good"  # Replace with your sentence
predicted_sentiment = predict_sentiment(sentence)
print(f"The predicted sentiment is: {predicted_sentiment}")


1/1 [==============================] - 0s 22ms/step
The predicted sentiment is: happy
